In [4]:
# establishing environment
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

print('Initalized')

Initalized


In [5]:
df = pd.read_csv('en-US.csv')
df.shape

(222186, 25)

In [6]:
df.columns

Index(['id', 'html', 'political', 'not_political', 'title', 'message',
       'thumbnail', 'created_at', 'updated_at', 'lang', 'images',
       'impressions', 'political_probability', 'targeting', 'suppressed',
       'targets', 'advertiser', 'entities', 'page', 'lower_page', 'targetings',
       'paid_for_by', 'targetedness', 'listbuilding_fundraising_proba',
       'page_id'],
      dtype='object')

In [7]:
df[df['political_probability'] < .1]['political_probability']

38        0.087742
58        0.095117
66        0.028502
183       0.016852
196       0.014723
            ...   
222007    0.028059
222084    0.021734
222128    0.033144
222129    0.014861
222135    0.026714
Name: political_probability, Length: 920, dtype: float64

In [8]:
df['paid_for_by']

0                                          Stop Republicans
1                  Planned Parenthood Federation of America
2                                             No Kid Hungry
3                                     Gillibrand for Senate
4                                                       NaN
                                ...                        
222181                 Shahid Buttar for Congress Committee
222182    CATHOLIC RELIEF SERVICES - UNITED STATES CONFE...
222183                               WARREN DEMOCRATS, INC.
222184                                                  NaN
222185                                                  NaN
Name: paid_for_by, Length: 222186, dtype: object

In [9]:
df.isnull().sum()

id                                     0
html                                   0
political                              0
not_political                          0
title                                 47
message                                0
thumbnail                              0
created_at                             0
updated_at                             0
lang                                   0
images                                 0
impressions                            0
political_probability                  0
targeting                          57057
suppressed                             0
targets                             3862
advertiser                         35855
entities                            3140
page                                5378
lower_page                          5378
targetings                         58272
paid_for_by                        59689
targetedness                       67357
listbuilding_fundraising_proba     74061
page_id         

In [10]:
df.dropna(subset=['title'], inplace= True)
df.drop(columns=['listbuilding_fundraising_proba', 'targetedness', 'targetings', 'targets', 
                 'targeting', 'images', 'lang', 'thumbnail', 'html', 'page_id', 'suppressed', 'entities', 'lower_page'], inplace=True)

df.isnull().sum()

id                           0
political                    0
not_political                0
title                        0
message                      0
created_at                   0
updated_at                   0
impressions                  0
political_probability        0
advertiser               35814
page                      5331
paid_for_by              59642
dtype: int64

In [11]:
df['socio_political_fb'] = df['paid_for_by'].notnull()


In [12]:
df['advert_by'] = np.where(df.paid_for_by.isnull(), df.advertiser, df.paid_for_by)


In [13]:
df[(df['advert_by'].isnull()) & (df['page'].notnull())][['page', 'advert_by', 'socio_political_fb']]

,page,advert_by,socio_political_fb
16997,https://www.facebook.com/indivisibleguide/,NaN,False
24074,https://www.facebook.com/clc.ctc/,NaN,False
29379,https://www.facebook.com/knightfdn/,NaN,False
33198,https://www.facebook.com/indivisibleguide/,NaN,False
34364,https://www.facebook.com/knightfdn/,NaN,False
...,...,...,...
192182,https://www.facebook.com/indivisibleguide/,NaN,False
192643,https://www.facebook.com/obamafoundation/,NaN,False
200395,https://www.facebook.com/indivisibleguide/,NaN,False
203419,https://www.facebook.com/indivisibleguide/,NaN,False


In [14]:
df.advert_by = np.where(df.advert_by.isnull(), df.page, df.advert_by)
df[(df['advert_by'].isnull()) & (df['advertiser'].notnull())]

,id,political,not_political,title,message,created_at,updated_at,impressions,political_probability,advertiser,page,paid_for_by,socio_political_fb,advert_by


In [15]:
df['advert_by'].fillna('unk', inplace=True)

In [16]:
df[(df['page']).notnull() & (df['advert_by'].isnull())]

,id,political,not_political,title,message,created_at,updated_at,impressions,political_probability,advertiser,page,paid_for_by,socio_political_fb,advert_by


In [17]:
df.shape

(222139, 14)

In [18]:
df.isnull().sum()

id                           0
political                    0
not_political                0
title                        0
message                      0
created_at                   0
updated_at                   0
impressions                  0
political_probability        0
advertiser               35814
page                      5331
paid_for_by              59642
socio_political_fb           0
advert_by                    0
dtype: int64

In [19]:
df.dropna(axis=1, inplace=True)
df.isnull().sum()

id                       0
political                0
not_political            0
title                    0
message                  0
created_at               0
updated_at               0
impressions              0
political_probability    0
socio_political_fb       0
advert_by                0
dtype: int64

In [20]:
df['message']

0         <p>Our elections are changing rapidly. Many De...
1         <p>Uncertain times shouldn’t impact your acces...
2         <p>Urgent! <a class="_58cn" href="https://www....
3         <p>If you want to take back the Senate and ret...
4         <span class="l_l7k1njrjc">Neck pain can be cau...
                                ...                        
222181    <p>Congress MUST act quickly to protect workin...
222182    <p>CRS is on the ground preventing and respond...
222183    <p>When she was running for president, one of ...
222184    <span class="g_rvp2lpj2y">Antioch’s MBA featur...
222185    <span class="u_pjn08f_5u">We’re all a little d...
Name: message, Length: 222139, dtype: object

In [21]:
from wrangle import wrangle

In [22]:
df = wrangle()
df.head()

,id,political,not_political,title,message,created_at,updated_at,impressions,political_probability,socio_political_fb,advert_by
0,23844966472130541,1,0,Stop Republicans,election changing rapidly many democrat terrif...,2020-04-28 18:29:51.893295+00,2020-04-28 20:00:18.291895+00,1,0.999563,True,Stop Republicans
1,23844607318170064,0,1,Planned Parenthood,uncertain time shouldnt impact access dependab...,2020-05-03 21:21:17.022063+00,2020-05-03 23:00:14.598025+00,1,0.426290,True,Planned Parenthood Federation of America
2,23844646560350048,0,0,No Kid Hungry,givingtuesdaya happening may 5th response covi...,2020-05-03 21:50:12.097628+00,2020-05-03 23:00:14.717141+00,1,0.625368,True,No Kid Hungry
3,23844775979230727,1,0,Kirsten Gillibrand,want take back senate retire mitch mcconnell s...,2020-04-27 23:44:32.620687+00,2020-04-28 19:48:27.917779+00,3,1.000000,True,Gillibrand for Senate
4,23844529028560107,0,2,Learn How to Eliminate Discomfort Today -->,neck pain caused several factor effectively tr...,2020-03-28 15:38:36.16756+00,2020-04-28 19:52:56.791781+00,3,0.701307,False,unk


In [23]:
df['message']

0         election changing rapidly many democrat terrif...
1         uncertain time shouldnt impact access dependab...
2         givingtuesdaya happening may 5th response covi...
3         want take back senate retire mitch mcconnell s...
4         neck pain caused several factor effectively tr...
                                ...                        
222181    housing homeless life working people lowincome...
222182    cr ground preventing responding covid19 outbre...
222183    running president one elizabeth favorite thing...
222184    antioch mba feature mutual importance people p...
222185                            little desperate together
Name: message, Length: 222139, dtype: object

In [1]:
from wrangle import wrangle
train, validate, test = wrangle('en-US.csv')
train.head()

,id,political,not_political,title,message,created_at,updated_at,impressions,political_probability,socio_political_fb,advert_by
132967,23842810769120558,0,0,End Citizens United,koch said spend 400 million preserve republica...,2018-03-01 19:50:38.892007+00,2018-03-01 19:50:38.910542+00,2,0.973059,False,Arizona Democratic Party
171238,hyperfeed_story_id_5cdb616c650cc2c08453510,1,0,Jay Inslee,simple get jay debate stage june need 65000 do...,2019-05-15 00:47:15.677809+00,2019-05-15 02:08:15.943151+00,1,0.999831,True,INSLEE FOR AMERICA
157472,6113998509676,0,0,Athens for Everyone,vote november 6 check voter guide information ...,2018-10-26 01:52:16.250883+00,2018-10-26 01:52:16.250883+00,1,0.984821,True,Athens for Everyone
18371,23843613746350544,2,2,PHS : Pennsylvania Horticultural Society,httpsbitly34h5kvxa,2019-09-30 08:21:12.454423+00,2019-10-01 17:05:33.831098+00,1,0.717423,False,PHS : Pennsylvania Horticultural Society
221209,23844574775710064,0,0,Planned Parenthood,planned parenthood expert make life easier eve...,2020-04-28 00:09:46.224171+00,2020-04-28 15:00:14.714633+00,1,0.617928,True,Planned Parenthood Federation of America
